In [6]:
import dimod
from dimod import ConstrainedQuadraticModel, Binary
from dimod import quicksum

# Depot, Customers 1 and 2, Vehicles a and b
nodes = [0, 1, 2]          # 0 = depot
customers = [1, 2]
vehicles = ['a', 'b']

# Time dictionary (cost)
t = {
    (0,1): 3, (1,0): 3,
    (0,2): 5, (2,0): 5,
    (1,2): 4, (2,1): 4
}


# Create binary decision variables                   
# ---------------------------------
x = {}
for i in nodes:
    for j in nodes:
        if i != j: 
            for k in vehicles:
                name = f"x_{i}_{j}_{k}"
                x[i,j,k] = Binary(name)


# Building the CQM
# -----------------
cqm = ConstrainedQuadraticModel()

# 1) Objective: minimize total travel time
objective = quicksum(t[i,j] * x[i,j,k]
                     for i,j,k in x.keys())
cqm.set_objective(objective)

# Constraints
# -------------

# Each customer visited once
for v in customers:
    cqm.add_constraint(
        quicksum(x[i,v,k] for i in nodes if i != v for k in vehicles) == 1,
        label=f"Visit_{v}"
    )

# Start and end at Depot for each vehicle
for k in vehicles:
    # start from depot once
    cqm.add_constraint(
        quicksum(x[0,j,k] for j in customers) == 1,
        label=f"Start_{k}"
    )
    # return to depot once
    cqm.add_constraint(
        quicksum(x[i,0,k] for i in customers) == 1,
        label=f"End_{k}"
    )

# Flow conservation
for v in customers:
    for k in vehicles:
        cqm.add_constraint(
            quicksum(x[i,v,k] for i in nodes if i != v) -
            quicksum(x[v,j,k] for j in nodes if j != v) == 0,
            label=f"flow_{v}_{k}"
        )


# Converting to BQM 
bqm, invert = dimod.cqm_to_bqm(cqm)

✅ CQM created with 10 constraints and 12 variables.
